In [232]:
import os
import subprocess
import sys
import json
import pandas as pd

In [233]:
run_iteration = 100
pwd = os.getcwd()
build_dir = os.path.join(pwd, "build")

In [234]:
def run(cmd, cwd=None) -> str:
    """Run a command and exit on failure (like set -e)."""
    print(f"+ {' '.join(cmd)}")
    result = subprocess.run(cmd, cwd=cwd, check=True, capture_output=True, text=True)
    return result.stdout

In [235]:
def analysis_code(key, data_tag) -> dict:
    binary_path = os.path.join(build_dir, f"power_meter_{key}")
    program_results = []
    if os.path.isfile(binary_path):
        print("\n" + "=" * 102)
        print(f"Analyzing for {key} ...")

        # Match bash loop: for i in {1..1}
        for i in range(0, run_iteration):
            if data_tag:
                data_file = os.path.join(
                    pwd, f"power_meter_{data_tag}_data.bin"
                )
                
                result = subprocess.run([binary_path, data_file], cwd=None, check=True, capture_output=True, text=True)
            else:
                result = subprocess.run([binary_path], cwd=None, check=True, capture_output=True, text=True)
            program_results.append(json.loads(result.stdout))


        # size binary
        result = run(["size", binary_path])
        lines = result.strip().splitlines()

        headers = lines[0].split()
        values = lines[1].split(maxsplit=len(headers) - 1)

        size_dict = dict(zip(headers, values))
        
        analysis_result = {"key":key,"program_results": program_results, "size_results": size_dict}
        return analysis_result

    else:
        raise ValueError(f"Error: Program {binary_path} not found!")

In [236]:
def analysis_deserialization(results: dict, id: str):
    rows = []

    overhead = results['overhead_deserialization']["program_results"]
    overhead_time_taken = sum(r["time_taken"] for r in overhead)/ len([r["time_taken"] for r in overhead])
    overhead_buffer_size = sum(r["buffer_size"] for r in overhead)/ len([r["buffer_size"] for r in overhead])
    overhead_flash_text = int(results['overhead_deserialization']['size_results']['text'])
    overhead_static_ram = (
                   int(results['overhead_deserialization']['size_results']["data"])
                    + int(results['overhead_deserialization']['size_results']["bss"])
                )
    for key, program in results.items():
        if id in key:
            times = [r["time_taken"] for r in program["program_results"]]
            buffers = [r["buffer_size"] for r in program["program_results"]]

            rows.append({
                "id": key,
                "avg_time_taken": sum(times) / len(times),
                "min_time": min(times),
                "max_time": max(times),
                "buffer_size": (sum(buffers) / len(buffers)),
                "flash_text": program["size_results"]["text"],
                "static_ram": (
                   int( program["size_results"]["data"])
                    + int(program["size_results"]["bss"])
                ),
                "avg_time_taken_net": (sum(times) / len(times)) - overhead_time_taken,
                "min_time_net": min(times) - overhead_time_taken,
                "max_time_net": max(times) - overhead_time_taken,
                "buffer_size_net": buffers[0] - overhead_buffer_size,
                "flash_text_net": int(program["size_results"]["text"]) - overhead_flash_text,
                "static_ram_net": (
                   int( program["size_results"]["data"])
                    + int(program["size_results"]["bss"])
                ) - overhead_static_ram,
            })

    df = pd.DataFrame(rows)
    # print(df)
    return df

            

In [237]:
def analysis_serialization(results: dict, id: str):
    rows = []

    overhead = results['overhead_serialization']["program_results"]
    overhead_time_taken = sum(r["time_taken"] for r in overhead)/ len([r["time_taken"] for r in overhead])
    overhead_buffer_size = sum(r["buffer_size"] for r in overhead)/ len([r["buffer_size"] for r in overhead])
    overhead_flash_text = int(results['overhead_serialization']['size_results']['text'])
    overhead_static_ram = (
                   int(results['overhead_serialization']['size_results']["data"])
                    + int(results['overhead_serialization']['size_results']["bss"])
                )
    for key, program in results.items():
        if id in key:
            times = [r["time_taken"] for r in program["program_results"]]
            buffers = [r["buffer_size"] for r in program["program_results"]]

            rows.append({
                "id": key,
                "avg_time_taken": sum(times) / len(times),
                "min_time": min(times),
                "max_time": max(times),
                "buffer_size": (sum(buffers) / len(buffers)),
                "flash_text": program["size_results"]["text"],
                "static_ram": (
                   int( program["size_results"]["data"])
                    + int(program["size_results"]["bss"])
                ),
                "avg_time_taken_net": (sum(times) / len(times)) - overhead_time_taken,
                "min_time_net": min(times) - overhead_time_taken,
                "max_time_net": max(times) - overhead_time_taken,
                "buffer_size_net": buffers[0] - overhead_buffer_size,
                "flash_text_net": int(program["size_results"]["text"]) - overhead_flash_text,
                "static_ram_net": (
                   int( program["size_results"]["data"])
                    + int(program["size_results"]["bss"])
                ) - overhead_static_ram,
            })

    df = pd.DataFrame(rows)
    # print(df)
    return df

            

In [238]:
program_results = {}
def main():
    # Configure and build
    run(["cmake", "-S", ".", "-B", "build"])
    run(["cmake", "--build", "build", "-j12"])

    # Program map (ordered like bash associative array is NOT guaranteed,
    # but Python 3.7+ dict preserves insertion order)
    programs = {
        "overhead_deserialization": "overhead",

        "nakedbytes_deserialization_without_access": "nakedbytes",
        "nakedbytes_deserialization_with_one_access": "nakedbytes",
        "nakedbytes_deserialization_with_all_access": "nakedbytes",

        "flatbuffers_deserialization_without_access": "flatbuffers",
        "flatbuffers_deserialization_with_one_access": "flatbuffers",
        "flatbuffers_deserialization_with_all_access": "flatbuffers",

        "protobuf_deserialization_without_access": "protobuf",
        "protobuf_deserialization_with_one_access": "protobuf",
        "protobuf_deserialization_with_all_access": "protobuf",

        "nanopb_deserialization_without_access": "protobuf",
        "nanopb_deserialization_with_one_access": "protobuf",
        "nanopb_deserialization_with_all_access": "protobuf",

        "overhead_serialization": "",

        "nakedbytes_serialization_primitives": "",
        "nakedbytes_serialization_all": "",

        "flatbuffers_serialization_primitives": "",
        "flatbuffers_serialization_all": "",

        "protobuf_serialization_primitives": "",
        "protobuf_serialization_all": "",

        "nanopb_serialization_primitives": "",
        "nanopb_serialization_all": "",
    }

    for key, data_tag in programs.items():
       program_results[key] = analysis_code(key=key, data_tag= data_tag)
    print(json.dumps(program_results,indent = 2 ))
       
       
    




In [239]:
main()

+ cmake -S . -B build
+ cmake --build build -j12

Analyzing for overhead_deserialization ...
+ size /home/busoye_tm/Documents/matthew/projects/nakedbytes/research_experiment/power_meter/build/power_meter_overhead_deserialization

Analyzing for nakedbytes_deserialization_without_access ...
+ size /home/busoye_tm/Documents/matthew/projects/nakedbytes/research_experiment/power_meter/build/power_meter_nakedbytes_deserialization_without_access

Analyzing for nakedbytes_deserialization_with_one_access ...
+ size /home/busoye_tm/Documents/matthew/projects/nakedbytes/research_experiment/power_meter/build/power_meter_nakedbytes_deserialization_with_one_access

Analyzing for nakedbytes_deserialization_with_all_access ...
+ size /home/busoye_tm/Documents/matthew/projects/nakedbytes/research_experiment/power_meter/build/power_meter_nakedbytes_deserialization_with_all_access

Analyzing for flatbuffers_deserialization_without_access ...
+ size /home/busoye_tm/Documents/matthew/projects/nakedbytes/re

In [240]:
analysis_deserialization(program_results, "deserialization_without_access")

,id,avg_time_taken,min_time,max_time,buffer_size,flash_text,static_ram,avg_time_taken_net,min_time_net,max_time_net,buffer_size_net,flash_text_net,static_ram_net
0,nakedbytes_deserialization_without_access,206.23,132,436,72.0,2781,680,23.03,-51.2,252.8,72.0,0,0
1,flatbuffers_deserialization_without_access,147.30,128,339,108.0,2781,680,-35.90,-55.2,155.8,108.0,0,0
2,protobuf_deserialization_without_access,20299.78,14335,41635,66.0,2738408,154364,20116.58,14151.8,41451.8,66.0,2735627,153684
3,nanopb_deserialization_without_access,49609.28,42023,75056,66.0,18390,744,49426.08,41839.8,74872.8,66.0,15609,64


In [241]:
analysis_deserialization(program_results, "deserialization_with_one_access")


,id,avg_time_taken,min_time,max_time,buffer_size,flash_text,static_ram,avg_time_taken_net,min_time_net,max_time_net,buffer_size_net,flash_text_net,static_ram_net
0,nakedbytes_deserialization_with_one_access,179.77,128,402,72.0,2823,680,-3.43,-55.2,218.8,72.0,42,0
1,flatbuffers_deserialization_with_one_access,231.93,163,876,108.0,2970,680,48.73,-20.2,692.8,108.0,189,0
2,protobuf_deserialization_with_one_access,16873.52,14047,26300,66.0,2738676,154364,16690.32,13863.8,26116.8,66.0,2735895,153684
3,nanopb_deserialization_with_one_access,51015.90,43644,74243,66.0,18550,744,50832.70,43460.8,74059.8,66.0,15769,64


In [242]:
analysis_deserialization(program_results, "deserialization_with_all_access")


,id,avg_time_taken,min_time,max_time,buffer_size,flash_text,static_ram,avg_time_taken_net,min_time_net,max_time_net,buffer_size_net,flash_text_net,static_ram_net
0,nakedbytes_deserialization_with_all_access,333.46,289,613,72.0,3173,680,150.26,105.8,429.8,72.0,392,0
1,flatbuffers_deserialization_with_all_access,667.69,538,1522,108.0,3549,680,484.49,354.8,1338.8,108.0,768,0
2,protobuf_deserialization_with_all_access,17300.92,14015,35185,66.0,2738840,154364,17117.72,13831.8,35001.8,66.0,2736059,153684
3,nanopb_deserialization_with_all_access,50507.97,43765,71665,66.0,18742,744,50324.77,43581.8,71481.8,66.0,15961,64


In [243]:
analysis_serialization(program_results, "serialization_primitive")

,id,avg_time_taken,min_time,max_time,buffer_size,flash_text,static_ram,avg_time_taken_net,min_time_net,max_time_net,buffer_size_net,flash_text_net,static_ram_net
0,nakedbytes_serialization_primitives,154.08,113,480,36.0,1974,632,11.73,-29.35,337.65,36.0,121,0
1,flatbuffers_serialization_primitives,6527.68,5703,9192,56.0,23342,728,6385.33,5560.65,9049.65,56.0,21489,96
2,protobuf_serialization_primitives,5557.72,4971,7619,33.0,2738580,154356,5415.37,4828.65,7476.65,33.0,2736727,153724
3,nanopb_serialization_primitives,32036.43,26104,49198,33.0,28185,712,31894.08,25961.65,49055.65,33.0,26332,80


In [244]:
analysis_serialization(program_results, "serialization_all")


,id,avg_time_taken,min_time,max_time,buffer_size,flash_text,static_ram,avg_time_taken_net,min_time_net,max_time_net,buffer_size_net,flash_text_net,static_ram_net
0,nakedbytes_serialization_all,137.86,122,295,71.0,1974,632,-4.49,-20.35,152.65,71.0,121,0
1,flatbuffers_serialization_all,7552.28,6562,11923,108.0,23694,728,7409.93,6419.65,11780.65,108.0,21841,96
2,protobuf_serialization_all,5538.47,4807,9376,66.0,2738708,154356,5396.12,4664.65,9233.65,66.0,2736855,153724
3,nanopb_serialization_all,49742.89,40845,87444,66.0,28265,712,49600.54,40702.65,87301.65,66.0,26412,80
